# Planning-Lab Lesson 1: Uninformed Search Strategies

In this first lesson we will work on Uninformed Search. 

### Maze Environments
The environments used is **SmallMaze** (visible in the figure).

<img src="images/maze.png" width="300">

The agent starts in cell $(0, 2)$ and has to reach the treasure in $(4, 3)$.

In order to use the environment we need first to import the packages of OpenAI Gym. Notice that due to the structure of this repository, we need to add the parent directory to the path

In [2]:
import os, sys, time, math

module_path = os.path.abspath(os.path.join('../tools'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.ai_lab_functions import *
import gym, envs
from sre_constants import FAILURE

## Assignment 1: Breadth-First Search (BFS)

Your first assignment is to implement the BFS algorithm on SmallMaze. In particular, you should implement both *tree search* and *graph search* versions of BFS that the generic bfs will call. 

The results returned by your **BFS** must be in the following form (path, time_cost, space_cost), more in detail:

- **path** - a tuple of state identifiers forming a path from the start state to the goal state. None if no solution is found.
- **time_cost** - the number of nodes checked during the exploration.
- **space_cost** - the maximum number of nodes in memory simultaneously.

**For the time_cost, we consider a node checked after its generation!**

After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- BFS_TreeSearch(problem)
- BFS_GraphSearch(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for *Graph Search* and *Tree Search*:

<img src="images/tree-graph-search.png" width="600">

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *BFS* algorithm, note that it refers to the implementation of the *Graph Search Version*:

<img src="images/BFS2.png" width="600">

**The next two functions have to be implemented:**

In [3]:
def BFS_TreeSearch(problem: gym.Env):
    """
    Tree Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    
    node = Node(problem.startstate, None)
    time_cost = 1
    space_cost = 1
    frontier = NodeQueue()
    frontier.add(node)
    goal = problem.goalstate
            
    while not frontier.is_empty():
        node = frontier.remove()
        if node.state == goal:
            return build_path(node), time_cost, space_cost
        else:
            for action in range(problem.action_space.n):
                child = Node(problem.sample(node.state, action), node)
                time_cost+=1
                if child.state == goal:
                    return build_path(child), time_cost, space_cost
                frontier.add(child)
                space_cost = max(space_cost, frontier.que_len)
    return FAILURE, time_cost, space_cost

In [4]:
def BFS_GraphSearch(problem: gym.Env):
    """
    Graph Search BFS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    time_cost = 1
    space_cost = 1
    frontier = NodeQueue()
    node = Node(problem.startstate, None)
    frontier.add(node)
    explored = set()
    goal = problem.goalstate
            
    if node.state == goal:
        return build_path(node), time_cost, space_cost
    
    while not frontier.is_empty():
        space_cost = max(space_cost, frontier.que_len+len(explored))
        node = frontier.remove()
        explored.add(node.state)
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state, action), node)
            time_cost+=1
            if child.state not in explored and child.state not in frontier:
                if child.state == goal:
                    return build_path(child), time_cost, space_cost
                frontier.add(child)
    return FAILURE, time_cost, space_cost       

**The following code calls your tree search and graph search version of BFS and checks the results**

In [5]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts = BFS_TreeSearch(environment)
solution_gs, time_gs, memory_gs = BFS_GraphSearch(environment)

results = CheckResult_L1A1([solution_ts, time_ts, memory_ts], [solution_gs, time_gs, memory_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  BFS TREE SEARCH PROBLEM  ########
##########################################
Your solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 103723
Max n° of nodes in memory: 77791

===> Your solution is correct!

##########################################
#######  BFS Graph SEARCH PROBLEM  #######
##########################################
Solution: [(0, 1), (0, 0), (1, 0), (2, 0), (3, 0), (4, 0), (4, 1), (4, 2), (4, 3)]
N° of nodes explored: 59
Max n° of nodes in memory: 15

===> Your solution is correct!



## Assignment 2:  Depth-Limited Search (DLS) and Iterative Deepening depth-first Search (IDS)

Your second assignment is to implement the IDS algorithm on SmallMaze. 
In particular, you are required to implement *DLS* in the *graph search* version, *DLS* in the *tree search* version, and the final *Iterative_DLS*.

Similarly to assignment 1, the results returned by your ids must be in the following form (path, Time Cost, Space Cost) described above. After the correctness of your implementations has been assessed, you can run the algorithms on the **SmallMaze** environment.

Functions to implement:

- Recursive_DLS_TreeSearch(node, problem, limit)
- Recursive_DLS_GraphSearch(node, problem, limit, explored)
- IDS(problem)

Function **build_path(node)** can be used to return a tuple of states from the root node (excluded) to another node by following parent links.

Here is the pseudo-code from the book **Artificial Intelligence: A Modern Approach** for the *Depth-Limited Search (Tree Search Version)* and *Iterative deepening depth-first search (Tree Search Version)*:
<img src="images/dls.png" width="600">
<img src="images/ids.png" width="600">

#### Note that Node() has a depthcost variable that represents the depth of the node in the search tree. This variable is automatically set by the Node constructor: if the root node has a depthcost of 0, its children will have a depthcost increased by 1. An example is shown below. The depthcost is useful to compute the space cost for IDS. An example is shown below:

In [6]:
start = environment.startstate
root = Node(start)  # parent = None and depthcost = 0 as default
child = Node(environment.sample(start, 0), root) # the depthcost is set automatically in the Node constructor
print("Root depthcost: {}\tChild depthcost: {}".format(root.depthcost, child.depthcost))

Root depthcost: 0	Child depthcost: 1


In [7]:
def DLS(problem, limit, RDLS_Function):
    """
    DLS
    
    Args:
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    node = Node(problem.startstate, None)
    return RDLS_Function(node, problem, limit, set())

**The next two functions have to be implemented:**

In [9]:
def Recursive_DLS_GraphSearch(node, problem, limit, explored):
    """
    Recursive DLS
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        explored: completely explored nodes
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    explored.add(node.state)
    space_cost = node.depthcost
    time_cost = 1 
    space_cost = node.depthcost
    time_cost = 1 
    if node.state == problem.goalstate:
        return result[0], time_cost, space_cost
    elif limit == 0:
        return "cut_off", time_cost, space_cost
    else:
        cutoff_occurred = False
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state, action), node)
            if child not in explored:
                explored.add(child.state)
                result = Recursive_DLS_GraphSearch(child, problem, limit-1, explored)
                if result == "cut_off":
                    cutoff_occurred = True
                elif result != "failure":
                    return result[0], time_cost, space_cost
        if cutoff_occurred:
            return "cut_off", time_cost, space_cost
        else:
            return "failure", time_cost, space_cost

In [10]:
def Recursive_DLS_TreeSearch(node, problem, limit, explored=None):
    """
    DLS (Tree Search Version)
    
    Args:
        node: node to explore
        problem: OpenAI Gym environment
        limit: depth limit for the exploration, negative number means 'no limit'
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
    space_cost = node.depthcost
    time_cost = 1 
    if node.state == problem.goalstate:
        return result[0], time_cost, space_cost
    elif limit == 0:
        return "cut_off", time_cost, space_cost
    else:
        cutoff_occurred = False
        for action in range(problem.action_space.n):
            child = Node(problem.sample(node.state, action), node)
            result = Recursive_DLS_TreeSearch(child, problem, limit-1)
            if result == "cut_off":
                cutoff_occurred = True
            elif result != "failure":
                return result[0], time_cost, space_cost
        if cutoff_occurred:
            return "cut_off", time_cost, space_cost
        else:
            return "failure", time_cost, space_cost

In [13]:
def IDS(problem, DLS_Function):
    """
    Iteartive_DLS DLS
    
    Args:
        problem: OpenAI Gym environment
        
    Returns:
        (path, time_cost, space_cost): solution as a path and stats.
    """
        
    total_time_cost = 0
    total_space_cost = 1
    for i in zero_to_infinity():
        result = DLS(problem, i, DLS_Function)
        if result != "cutoff":
            return result, total_time_cost, total_space_cost, i

**The following code calls your version of IDS and checks the results:**

In [16]:
envname = "SmallMaze-v0"
environment = gym.make(envname)

solution_ts, time_ts, memory_ts, iterations_ts = IDS(environment, Recursive_DLS_TreeSearch)
solution_gs, time_gs, memory_gs, iterations_gs = IDS(environment, Recursive_DLS_GraphSearch)

results = CheckResult_L1A2([solution_ts, time_ts, memory_ts, iterations_ts], [solution_gs, time_gs, memory_gs, iterations_gs], environment)
results.check_sol_ts()
results.check_sol_gs()

##########################################
#######  IDS TREE SEARCH PROBLEM  ########
##########################################
Necessary Iterations: 0


KeyboardInterrupt: 

### Discussion

Now that you have correctly implemented both BFS and IDS what can you say about the solutions they compute? Are there significant differences in the stats?